In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from enum import Enum, auto

In [ ]:
import preprocessing
df = preprocessing.df
categories = preprocessing.categories

In [ ]:
# setkeys = pd.DataFrame(preprocessing.setkeys, columns=["key", "value"])
# cat_sizes = setkeys["key"].value_counts()
# c = cat_sizes.to_numpy()
# w = 1 / np.sqrt(c)
# dict(zip(cat_sizes.index, w / np.sum(w)))

### Show all alternative values

In [ ]:
# altcols = [col for col in df.columns if col.endswith("_alt")]
# print("\nAll countries with alternative values:")
# display(df[df[altcols].applymap(len).sum(axis=1) > 0])

In [ ]:
class DifficultyLevel(Enum):
    EASY = "easy"
    MEDIUM = "medium"
    HARD = "hard"
    
    def __str__(self):
        return self.value
    
    def __repr__(self):
        return str(self)

In [ ]:
def save_games(games, name: str, difficulty_level=None):
    timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    info = [timestamp, name, lang]
    if difficulty_level is not None:
        info.append(str(difficulty_level).lower())
    path = f"../../data/games-{'-'.join(info)}.json"
    json.dump([game.to_json() for game in games], open(path, mode="w", encoding="utf-8"))
    print(f"{len(games)} games saved to {path}")

In [ ]:
from game import Constraint, GameGenerator

constraints = [
    # Some categories are pretty boring to appear multiple times
    Constraint.category_at_most("capital_ending_letter", 1),
    Constraint.category_at_most("capital_starting_letter", 1),
    Constraint.category_at_most("ending_letter", 1)
]
category_probs = {
    'continent': 4,
    'starting_letter': 3,
    'ending_letter': 1.5,
    'capital_starting_letter': 2,
    'capital_ending_letter': .5,
    'flag_colors': 3,
    'landlocked': 4,
    'island': 4
}

In [ ]:
# generator = preprocessing.get_generator(constraints, category_probs, field_size=3,
#                                         seed=None, selection_mode="shuffle_setkeys", uniform=False, shuffle=True)
# games = generator.sample_games(n=1000)
# save_games(games, "2-more-categories")

In [ ]:
from importlib import reload
reload(preprocessing)

In [ ]:
generator1 = preprocessing.get_generator(constraints, category_probs, field_size=3,
                                         seed=None, selection_mode="shuffle_setkeys", uniform=False, shuffle=True)
games1 = list(generator1.sample_games(n=500))

# Count category occurences
print(pd.Series(sum([[cat.key for cat, value in g.rows + g.cols] for g in games1], [])).value_counts())

generator2 = preprocessing.get_generator(constraints, category_probs, field_size=3,
                                         seed=None, selection_mode="shuffle_categories", uniform=False, shuffle=True)
games2 = list(generator2.sample_games(n=500))

# Count category occurences
print(pd.Series(sum([[cat.key for cat, value in g.rows + g.cols] for g in games2], [])).value_counts())

In [ ]:
for game in games[:20]:
    display(game.to_dataframe(solution=True))

# Compute country difficulties

In [ ]:
def normalize_zero_centered(x: pd.Series, scale=1):  # map to interval [-scale, +scale]
    return scale * (2 * (x - x.min()) / (x.max() - x.min()) - 1)

def normalize(x: pd.Series, scale=1):  # map to interval [0, scale]
    return scale * (x - x.min()) / (x.max() - x.min())

# def normalize_levels(x: pd.Series, scale=10):
#     return np.ceil(normalize(x, scale))

# def sigmoid(x: pd.Series):
#     return 1 / (1 + np.exp(-x))

# def normalize_levels_logistic(x: pd.Series, scale=10):
#     return normalize_levels(sigmoid(normalize_zero_centered(x)), scale)

In [ ]:
df.loc[df["name"] == "Monaco", "difficulty"].iloc[0]

In [ ]:
sqrt_name_length = normalize(np.sqrt(df["name"].apply(len)), scale=10)
log_population = normalize(np.log(df["population"]), scale=10)
log_gdp = normalize(np.log(df["gdp"]), scale=10)
log_gdp_per_capita = normalize(np.log(df["gdp_per_capita"]), scale=10)


# df["difficulty"] = normalize_levels_logistic((1 * log_population + 2 * log_gdp - .1 * sqrt_name_length) * (-1), scale=10)
df["difficulty"] = normalize((1 * log_population + 2 * log_gdp + .5 * log_gdp_per_capita - .25 * sqrt_name_length) * (-1), scale=10)

# VATICAN is an outlier. As reference, set as difficult as MONACO.
df.loc[df["name"] == "Vatican", "difficulty"] = df.loc[df["name"] == "Monaco", "difficulty"].iloc[0]

# df1 = df.nsmallest(50, "difficulty").reset_index()
# df1
# df1[df1["landlocked"]][["iso", "difficulty", "gdp", "population"]]
# df.nlargest(20, "difficulty")[["name", "gdp", "population", "difficulty"]]
df[["name", "gdp", "gdp_per_capita", "population", "difficulty"]].sort_values("difficulty").head(50)

In [ ]:
categories = preprocessing.categories
setkeys = preprocessing.setkeys

In [ ]:
items = df[["iso", "name", "difficulty"]].copy()
for key, value in setkeys:
    cat = categories[key]
    items[(key, value)] = items.iso.apply(lambda iso: iso in cat.sets.loc[value])

In [ ]:
info = pd.Series({(key, value): items[items[(key, value)]].difficulty.agg(list) for key, value in setkeys})
info = info.reset_index()
info.columns=["key", "value", "difficulties"]
info["cat_difficulty"] = normalize(info.key.apply(lambda key: categories[key].difficulty), scale=10)
info["size"] = info.difficulties.apply(len)
info["size_score"] = normalize(-np.sqrt(info["size"]), scale=10)
info["mean"] = info.difficulties.apply(np.mean)
info["min"] = info.difficulties.apply(min)
info["max"] = info.difficulties.apply(max)
info["median"] = info.difficulties.apply(np.median)
info["std"] = info.difficulties.apply(np.std)
info.drop(columns=["difficulties"], inplace=True)
info["countries"] = info.apply(lambda row: categories[row["key"]].sets.loc[row["value"]], axis=1)

info["difficulty"] = normalize(3 * info["median"] + .5 * info["std"] + 2 * info["size_score"] + 1.5 * info["cat_difficulty"], scale=10)
info.nsmallest(20, "difficulty")

In [ ]:
info.nlargest(20, "mean")

In [ ]:
info[info["key"] == "flag_colors"].sort_values("std")

In [ ]:
# Calculate difficulty of continents

data = df[["iso", "name", "continent", "difficulty"]].copy()
data.rename(columns={"continent": "category"}, inplace=True)

# Step 1: Calculate average difficulty per category
# Step 2: Calculate standard deviation per category
# Step 3: Adjust for category size (you can adjust the value to your liking)
categories = pd.DataFrame({"avg": data.groupby('category')['difficulty'].mean(),
                           "median": data.groupby('category')['difficulty'].median(),
                           "std": data.groupby('category')['difficulty'].std(),
                           "size": data['category'].value_counts()})

# Idea: entropy (europe is easy because oceania is hard)

# Step 4: Combine
# Step 5: Normalize
categories["difficulty"] = normalize(2 * categories["avg"] + categories["std"] + np.sqrt(categories["size"]), scale=10)

categories.sort_values("difficulty")

In [ ]:
def entropy(x: pd.Series) -> float:
    p = x.value_counts() / len(x)
    return np.sum(-p * np.log2(p))

def entropies(x: pd.Series, y: pd.Series):
    pass
#     return pd.concat([x, y], axis=1).groupby(x.name)[y.name].agg(entropy)


# entropy(pd.Series(np.array([1,2,3,4])))
# entropy(pd.Series(np.array([1,1,1,1])))
# entropies(df["continent"], df["difficulty"])

In [ ]:
import matplotlib.pyplot as plt

# plt.scatter(x=np.log(df["gdp"]) - np.log(df["population"]), y=df["gdp_per_capita"])
# plt.scatter(x=np.log(df["population"]), y=np.log(df["gdp"]), c=df["difficulty"])
# plt.scatter(x=np.log(df["population"]), y=np.log(df["gdp"]), c=df["difficulty"])
plt.show()

In [ ]:
def get_difficulty(game):
    game_cats = [cat for cat, val in game.rows + game.cols]
    cat_sizes = [len(cat.sets) for cat in game_cats]
    
    solutions = sum([[cell for cell in row] for row in game.solutions], [])
    alt_solutions = sum([[cell for cell in row] for row in game.alt_solutions], [])
    all_solutions = list(sorted(set(sum(solutions + alt_solutions, []))))
    cell_sizes = list(map(len, solutions))
    num_unique = len([i for i, size in enumerate(cell_sizes) if size == 1])
    cat_difficulty = sum(cat.difficulty for cat in game_cats)
    
    return {
        "cell_sizes": cell_sizes,
        "cell_sizes_median": pd.Series(cell_sizes).median(),
        "cell_min_difficulties": [df[df["iso"].isin(cell)]["difficulty"].min() for cell in solutions],
        "cell_median_difficulties": [df[df["iso"].isin(cell)]["difficulty"].median() for cell in solutions],
        "num_unique": num_unique,
        "max_num_cells": max(len([cell for cell in solutions if c in cell]) for c in all_solutions),
        "max_num_cells_unique_solution": max(len([cell for cell in solutions if len(cell) == 1 and c in cell]) for c in all_solutions),
        "cat_sizes": cat_sizes,
        "all_solutions": len(all_solutions),
        "cat_sizes_median": pd.Series(cell_sizes).median(),
        "cat_difficulty": cat_difficulty / len(game_cats)
    }

# info = pd.DataFrame([{"game": game, **get_difficulty(game)} for game in games])
# info["score"] = -info["num_unique"] + info["cat_difficulty"] + info["cell_sizes_median"] + info["cat_sizes_median"]

info["cell_max_min_difficulties"] = info["cell_min_difficulties"].apply(max)


# info.head(30)

# idea: countries with small population / size are harder to guess
# log(sum(populations of all solution)) median of all cells

In [ ]:
info

In [ ]:
plt.hist(df["difficulty"], bins=20, rwidth=.9)
plt.title("Country difficulties")
plt.show()

In [ ]:
plt.hist(log_name_length, bins=15, rwidth=.9)
plt.title("Country name length")
plt.show()

In [ ]:
plt.hist(info["cell_max_min_difficulties"], bins=20, rwidth=.9)
plt.title("Games: Hardest cell solution difficulty")
plt.show()

In [ ]:
plt.hist(info["num_unique"], bins=range(10), rwidth=.9)
plt.title("Games: Number of unique-solution cells")
plt.show()

In [ ]:
info["difficulty"] = 5 * info["cell_max_min_difficulties"] + 2 * info["num_unique"] + 5 * info["cat_difficulty"] + .5 * info["cell_sizes_median"] + .5 * info["cat_sizes_median"]
info["difficulty"] = 10 * ((info["difficulty"] - info["difficulty"].min()) / (info["difficulty"].max() - info["difficulty"].min()))

easiest = [game for i, game in enumerate(games) if i in info.nsmallest(20, "difficulty").index.tolist()]

for game in easiest:
    display(game.to_dataframe(solution=True))
    
json.dump([game.to_json() for game in easiest], open("../../data/games_easy.json", mode="w", encoding="utf-8"))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(12,3))
plt.subplot(121)

plt.hist(info["max_num_cells"], rwidth=.9, bins=range(10))
plt.xticks(np.arange(.5,10,1), labels=range(10))
plt.xlim([-.5,9.5])
# plt.xticklabels()
plt.title("Countries appear in up to x cells")


plt.subplot(122)

plt.hist(info["max_num_cells_unique_solution"], rwidth=.9, bins=range(10))
plt.xticks(np.arange(.5,10,1), labels=range(10))
plt.xlim([-.5,9.5])
# plt.xticklabels()
plt.title("Countries appear in up to x cells as unique solution")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.hist(info["cat_difficulty"], rwidth=.9)
plt.title("Category difficulty sum")
plt.show()

In [ ]:
games[0].to_json()

In [ ]:
# save()